In [50]:
import pandas as pd

In [51]:
computational_df= pd.read_csv('comp.csv')
computational_df.head()

,formula,Property band_gap,Property band_gap units,Property SMILES
0,GdCoO$_{3}$,0.1754,eV/atom,NaN
1,PrPt,0.0000,eV/atom,NaN
2,Tl$_{2}$SeO$_{4}$,3.3122,eV/atom,NaN
3,Ba$_{5}$(InSb$_{3}$)$_{2}$,0.0000,eV/atom,NaN
4,NiP$_{2}$,0.5442,eV/atom,NaN


In [52]:
computational_df.rename(columns={"formula":"chemical_formula","Property band_gap":"band_gap"},inplace=True,errors="raise")

In [53]:
computational_df['chemical_formula']=computational_df['chemical_formula'].astype('str').apply(lambda x:x.translate
                                          ({ord(c): None for c in "_${}"}))

In [54]:
from matminer.featurizers.conversions import StrToComposition
computational_df = StrToComposition().featurize_dataframe(computational_df, 'chemical_formula')

In [55]:
from matminer.featurizers.composition import ElementProperty

ep_feat = ElementProperty.from_preset(preset_name="magpie")
computational_df = ep_feat.featurize_dataframe(computational_df, col_id="composition",ignore_errors=True,return_errors=True)  # input the "composition" column to the featurizer
computational_df.head()

,chemical_formula,band_gap,Property band_gap units,Property SMILES,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,...,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,ElementProperty Exceptions
0,GdCoO3,0.1754,eV/atom,NaN,"(Gd, Co, O)",8.0,64.0,56.0,23.000000,18.000000,...,0.309694,0.495511,0.0,12.0,194.0,182.0,84.800000,87.360000,12.0,NaN
1,PrPt,0.0000,eV/atom,NaN,"(Pr, Pt)",59.0,78.0,19.0,68.500000,9.500000,...,0.000000,0.000000,0.0,194.0,225.0,31.0,209.500000,15.500000,194.0,NaN
2,Tl2SeO4,3.3122,eV/atom,NaN,"(Tl, Se, O)",8.0,81.0,73.0,32.571429,28.081633,...,0.000000,0.000000,0.0,12.0,194.0,182.0,64.285714,74.122449,12.0,NaN
3,Ba5(InSb3)2,0.0000,eV/atom,NaN,"(Ba, In, Sb)",49.0,56.0,7.0,52.615385,2.603550,...,0.000000,0.000000,0.0,139.0,229.0,90.0,186.076923,33.017751,166.0,NaN
4,NiP2,0.5442,eV/atom,NaN,"(Ni, P)",15.0,28.0,13.0,19.333333,5.777778,...,0.198465,0.264620,0.0,2.0,225.0,223.0,76.333333,99.111111,2.0,NaN


In [120]:
len(X)

40317

In [121]:
a=X.dropna()
len(a)

40311

In [116]:
computational_df.loc[5644]['MagpieData minimum MeltingT']

nan

In [117]:
computational_df.loc[5644]

chemical_formula                            He
band_gap                               17.8914
Property band_gap units                eV/atom
Property SMILES                            NaN
composition                               (He)
                                        ...   
MagpieData range SpaceGroupNumber            0
MagpieData mean SpaceGroupNumber           225
MagpieData avg_dev SpaceGroupNumber          0
MagpieData mode SpaceGroupNumber           225
ElementProperty Exceptions                 NaN
Name: 5644, Length: 138, dtype: object

In [122]:
excluded = ["chemical_formula","band_gap","Property band_gap units", "Property SMILES", "composition","ElementProperty Exceptions"]
excluded_df = computational_df.drop(excluded,axis=1)
X= excluded_df.dropna()
y=computational_df['band_gap'].values

In [124]:
excluded = [ "Property SMILES", "ElementProperty Exceptions"]
excluded_df = computational_df.drop(excluded,axis=1).dropna()
X = excluded_df.drop(["chemical_formula","band_gap","Property band_gap units","composition"],axis=1)
y= excluded_df['band_gap']

In [139]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import numpy as np
min_max_scaler = preprocessing.StandardScaler()
X_train_minmax = min_max_scaler.fit_transform(X)
lr = LinearRegression()

lr.fit(X_train_minmax, y)

# get fit statistics
print('training R2 = ' + str(round(lr.score(X_train_minmax, y), 3)))
print('training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=lr.predict(X_train_minmax))))

training R2 = 0.605
training RMSE = 1.100


In [144]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=1)

rf.fit(X, y)
print ('R2 = ' + str(round(rf.score(X, y), 3)))
print ('RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=rf.predict(X))))

R2 = 0.963
RMSE = 0.337
